In [1]:
import pandas as pd
from pprint import pprint
#from caplot import Pca

In [2]:
df = pd.read_csv('data/samples.tsv.gz', sep='\t')

In [3]:
pprint(list(df.columns))

['s',
 'pheno-subpopulation',
 'pheno-superpopulation',
 'pheno-age',
 'pheno-t2d',
 'pheno-bmi',
 'pheno-isFemale',
 'sample_qc-call_rate',
 'sample_qc-n_called',
 'sample_qc-n_not_called',
 'sample_qc-n_filtered',
 'sample_qc-n_hom_ref',
 'sample_qc-n_het',
 'sample_qc-n_hom_var',
 'sample_qc-n_non_ref',
 'sample_qc-n_singleton',
 'sample_qc-n_snp',
 'sample_qc-n_insertion',
 'sample_qc-n_deletion',
 'sample_qc-n_transition',
 'sample_qc-n_transversion',
 'sample_qc-n_star',
 'sample_qc-r_ti_tv',
 'sample_qc-r_het_hom_var',
 'sample_qc-r_insertion_deletion',
 'pcaSS1-scores_1',
 'pcaSS1-scores_2',
 'pcaSS1-scores_3',
 'pcaSS2-scores_1',
 'pcaSS2-scores_2',
 'pcaSS2-scores_3',
 'pcaSS2-scores_4',
 'pcaSS2-scores_5',
 'pcaSS2-scores_6',
 'pcaSS2-scores_7',
 'pcaSS2-scores_8',
 'pcaSS2-scores_9',
 'pcaSS2-scores_10',
 'pcaMAF-scores_1',
 'pcaMAF-scores_2',
 'pcaMAF-scores_3',
 'pcaMAF-scores_4',
 'pcaMAF-scores_5',
 'pcaMAF-scores_6',
 'pcaMAF-scores_7',
 'pcaMAF-scores_8',
 'pcaMAF-sco

In [4]:
df

,s,pheno-subpopulation,pheno-superpopulation,pheno-age,pheno-t2d,pheno-bmi,pheno-isFemale,sample_qc-call_rate,sample_qc-n_called,sample_qc-n_not_called,...,pca-scores_11,pca-scores_12,pca-scores_13,pca-scores_14,pca-scores_15,pca-scores_16,pca-scores_17,pca-scores_18,pca-scores_19,pca-scores_20
0,HG00096,GBR,EUR,43,0,25.4240,False,1.0,81207,0,...,0.002360,-0.007440,-0.018326,-0.011122,-0.002118,0.016371,-0.001327,0.009382,0.015766,0.016529
1,HG00097,GBR,EUR,77,0,22.7940,True,1.0,81207,0,...,-0.021346,0.000352,0.011478,0.002156,-0.001305,0.010297,0.005677,-0.004408,-0.000739,0.008136
2,HG00099,GBR,EUR,93,1,24.4820,True,1.0,81207,0,...,-0.007263,0.001839,-0.007027,0.017851,0.008086,0.007488,-0.006006,0.011793,-0.002705,0.017171
3,HG00100,GBR,EUR,49,0,25.4670,True,1.0,81207,0,...,-0.010677,0.005735,0.004658,-0.011409,-0.009111,0.007221,-0.006345,0.002117,0.004292,-0.004159
4,HG00101,GBR,EUR,37,0,26.2730,False,1.0,81207,0,...,-0.003234,0.002436,-0.017813,0.002750,-0.006098,-0.012314,0.014546,-0.006744,0.004415,0.002690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,NA21137,GIH,SAS,72,1,10.0490,True,1.0,81207,0,...,-0.007564,-0.002063,0.000601,0.000591,0.005463,0.007928,0.003090,-0.003936,0.013568,-0.001185
2500,NA21141,GIH,SAS,87,0,5.9742,True,1.0,81207,0,...,0.001109,0.005649,0.002637,0.000618,0.003520,0.004999,0.004118,-0.001062,-0.001690,-0.000253
2501,NA21142,GIH,SAS,89,1,8.6607,True,1.0,81207,0,...,-0.000806,0.005311,0.008220,-0.002050,-0.006598,-0.001819,-0.001951,-0.004965,-0.004919,0.002718
2502,NA21143,GIH,SAS,59,0,8.1004,True,1.0,81207,0,...,0.000982,-0.014256,0.001442,0.009168,0.004654,-0.000056,0.004146,-0.002252,-0.007060,-0.007535
